# PySCF Practice

In [1]:
from pyscf import gto, scf, dft
import numpy as np
from IPython.display import clear_output
import pandas as pd
from pathlib import Path

## Creaing a molecule
`gto.M` is a short cut function to initialize a molecule; you can set the basis or the symmetry

In [10]:
mol_h2o = gto.M(atom = 'O 0 0 0; H 0 1 0; H 0 0 1', basis = 'sto-3g')
mol_n3 = gto.M(atom = 'N 0 0 0; N 0 0 1; N 0 0 2', basis='sto-3g', charge=-1)
mol_n3_bent = gto.M(atom='N 0 0 0; N 0 1 1; N 0 1 0.5', basis='sto-3g', charge=-1)

`Mole` offers more customized intialisation. 

## Mean-Field theory
e.g. Hartree-fock (also called self-consistent field method (SCF))
### Package Options
- RHF (Restricted HF) - for closed-shell molecules with no unpaired e-. (idea: use the same equation to describe both electrons)
- UHF (Unrestricted HF) - for open-shell molecules, e.g. metals, radicals, ...
- ROHF (Restricted Open-Shell HF) - hybrid, keep the spin symmetry correct.
### Basis options

In [21]:
rhf_h2o = scf.RHF(mol_h2o)
e_h2o = rhf_h2o.kernel()

rhf_n3 = scf.RHF(mol_n3)
rhf_n3_bent = scf.RHF(mol_n3_bent)
e_n3 = rhf_n3.kernel()
e_n3_bent = rhf_n3_bent.kernel()

converged SCF energy = -74.9611711378677
converged SCF energy = -160.662076594209
converged SCF energy = -153.313769835814


**Going deep into SCF**: Extracting fock matrix and density matrix

In [ ]:
from pyscf import gto, scf
from pyscf.lo import orth
import numpy as np

# Grab SAAO coefficients from a molecule
S = rhf_h2o.get_ovlp()                               # AO overlap (nao×nao)
C_sao = orth.orth_ao(mol_h2o, method='lowdin', s=S)     # Symmetric orthogonalizer: for transformation to SAAO basis

# Obtain the AO basis matrices:
H_ao  = rhf_h2o.get_hcore()                              # Core Hamiltonian in AO basis
F_ao  = rhf_h2o.get_fock(dm=rhf_h2o.make_rdm1())              # Fock in AO basis
P_ao  = rhf_h2o.make_rdm1()                              # Density in AO basis
J_ao, K_ao = rhf_h2o.get_jk(mol_h2o, dm=rhf_h2o.make_rdm1())

# Project into SAAO basis:
H_sao = C_sao.T @ H_ao @ C_sao
F_sao = C_sao.T @ F_ao @ C_sao
P_sao = C_sao.T @ P_ao @ C_sao
J_sao = C_sao.T @ J_ao @ C_sao
K_sao = C_sao.T @ K_ao @ C_sao

print("SAAO coefficients shape:", C_sao.shape)
print("SAAO Fock matrix shape:", F_sao.shape)
print("SAAO Density matrix shape:", P_sao.shape)
print("SAAO Core Hamiltonian shape:", H_sao.shape)
print("SAAO Coulomb matrix shape:", J_sao.shape)
print("SAAO Exchange matrix shape:", K_sao.shape)


diag_blocks = []
atom_slices = mol_h2o.aoslice_by_atom()
for (_, _, ist, sto) in atom_slices:
    diag_blocks.append(np.stack([
      np.diag(F_sao[ist:sto, ist:sto]),
      np.diag(J_sao[ist:sto, ist:sto]),
      np.diag(K_sao[ist:sto, ist:sto]),
      np.diag(P_sao[ist:sto, ist:sto]),
      np.diag(H_sao[ist:sto, ist:sto]),
    ], axis=1))

# 2) Pad each block to (max_n,5)
max_n = max(b.shape[0] for b in diag_blocks)
padded = []
for b in diag_blocks:
    pad = np.zeros((max_n - b.shape[0], 5), dtype=b.dtype)
    padded.append(np.vstack([b, pad]))

print(np.array(padded).shape)  # (n_atoms, max_n, 5)



SAAO coefficients shape: (7, 7)
SAAO Fock matrix shape: (7, 7)
SAAO Density matrix shape: (7, 7)
SAAO Core Hamiltonian shape: (7, 7)
SAAO Coulomb matrix shape: (7, 7)
SAAO Exchange matrix shape: (7, 7)
(3, 5, 5)


Default spin is 0. For molecules with unpaired electrons, spin number must be updated; otherwise there will be error.

In [4]:
mol_no2 = gto.M(atom = 'N 0 0 0; O 0 1 0; O 0 0 1', basis='sto-3g', spin=1)
uhf_no2 = scf.UHF(mol_no2)
e_no2 = uhf_no2.kernel()


rohf_no2 = scf.ROHF(mol_no2)
e_no2_ro = rohf_no2.kernel()

converged SCF energy = -200.630088281403  <S^2> = 0.75238228  2S+1 = 2.0023809
converged SCF energy = -200.628674251033


For fun, here is a cubane molecule:

In [5]:
mol = gto.M(
    atom = '''
C   0.785   0.785   0.785
C  -0.785   0.785   0.785
C   0.785  -0.785   0.785
C   0.785   0.785  -0.785
C  -0.785  -0.785   0.785
C  -0.785   0.785  -0.785
C   0.785  -0.785  -0.785
C  -0.785  -0.785  -0.785
H   1.420   1.420   1.420
H  -1.420   1.420   1.420
H   1.420  -1.420   1.420
H   1.420   1.420  -1.420
H  -1.420  -1.420   1.420
H  -1.420   1.420  -1.420
H   1.420  -1.420  -1.420
H  -1.420  -1.420  -1.420
''',
    unit   = 'Angstrom',
    charge = 0,
    spin   = 0,
    basis  = 'sto-3g',   
)
rhf = scf.RHF(mol)
e = rhf.kernel()

converged SCF energy = -303.77968305976


### What exactly is it calculating here?


## Density Functional Theory
The same kinds of methods available:
- RKS
- UKS
- ROKS

The customisation here is **exchange-correlation** and also **basis**

In [6]:
from pyscf import dft
rks_h2o = dft.RKS(mol_h2o)
rks_h2o.xc = 'b3lyp'

e_rks = rks_h2o.kernel()

/home/yizhou_chen/miniconda3/envs/pyscf-env/lib/python3.12/site-packages/pyscf/dft/libxc.py:512: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


converged SCF energy = -75.3186721647241


## A little on quantum mechanics
Average total energy for a state = expected value of H : $E[\Psi] = \int \Psi^* \hat{H} \Psi dr \equiv \left<\Psi | \hat{H} | \Psi \right> $
- Note this notation is consistent with inner product notation: three items just means $\Psi*H*\Psi$
- QM postulate.

### Hartree-fock Approximation:
Wavefunction is the determinante of functions of every electron $\phi$: 

$\Psi = \frac{1}{\sqrt{N!}}*\text{det}[\phi_1\phi_2...\phi_N]$



## Time-depenedent mean-field theory

In [7]:
bond_len = 0.9572
bond_ang = 104.5
x = bond_len * np.sin(np.radians(bond_ang))
y = bond_len * np.cos(np.radians(bond_ang))


h2o_atoms = ('O','H','H') # what atom that each row corresponds to 
h2o_coords = np.array([[0,0,0],[x, y, 0],[-x,y,0]]) # put in the actual data

def np_to_scfinput(atoms, coords):
    '''
    Input: tuple of (coordinates: np.array; atom representation: tuple of strings)
    '''
    # safeguarding here
    # convertion
    raw = []
    for atom, coord in zip(atoms, coords):
        raw.append(' '.join([atom, *coord.astype(str)]))
    
    return ' ;'.join(raw)

mol_h2o = gto.M(atom=np_to_scfinput(h2o_atoms,h2o_coords), basis='def2-SVP')

In [8]:
from pyscf import tdscf
rhf_h2o = scf.RHF(mol_h2o)
rhf_h2o.kernel()
tdhf_h2o = tdscf.TDHF(rhf_h2o)
tdhf_h2o.nstates = 6
tdhf_h2o.kernel()
tddft_h2o = tdscf.TDA(rks_h2o) # TDDFT with Tamm-Dancoff approximation
tddft_h2o.nstates = 4
tddft_h2o.kernel()

converged SCF energy = -75.924388049698
Excited State energies (eV)
[ 8.63508502  9.12634776 10.86047437 11.30969812 16.79501776 19.70091211]
Excited State energies (eV)
[11.01533717 12.31203257 15.22979478 17.120181  ]


(array([0.40480618, 0.45245886, 0.55968464, 0.62915505]),
 [(array([[ 0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00],
          [-3.15040943e-16,  0.00000000e+00],
          [ 7.07106781e-01,  8.64470828e-15]]),
   0),
  (array([[-5.76459183e-19,  6.07422976e-18],
          [ 7.29776272e-17, -1.89745659e-18],
          [-3.57225550e-16, -2.91314995e-17],
          [-3.81740742e-16,  5.33317334e-16],
          [ 9.03263152e-15, -7.07106781e-01]]),
   0),
  (array([[ 3.34988299e-04,  1.18459613e-17],
          [-2.85327320e-02,  1.03233934e-15],
          [-8.10530918e-15, -2.53239935e-01],
          [-6.59587224e-01, -2.59874467e-15],
          [-2.93869309e-16,  3.68710777e-16]]),
   0),
  (array([[-3.12292190e-18,  8.08219822e-04],
          [ 1.41671558e-15,  3.88443266e-02],
          [-4.44571959e-01,  1.26528300e-14],
          [ 2.74919046e-15, -5.48494520e-01],
          [-7.61189056e-18, -1.36425086e-

In [9]:
weights, nto = tdhf_h2o.get_nto(state=2)
print(nto)

[[ 5.03218838e-02  1.31595049e-13 -3.27635787e-13  2.91692939e-01
   9.87234838e-01  1.40435442e-01  6.76383448e-14 -2.58177322e-14
  -3.02705204e-01  3.18804866e-02 -1.32172933e-02 -1.12972506e-01
  -3.44528499e-02 -9.08597780e-14  3.74331452e-03  2.29268622e-02
   2.30975245e-04  1.12729202e-01  1.01430677e-03 -6.15018897e-03
  -2.39158945e-02  4.59032558e-14 -1.93429871e-02  8.16927429e-02]
 [ 1.05273995e-01  3.13443726e-13 -6.44782938e-13  5.47257364e-01
  -3.96139724e-02  2.84957683e-01  7.02068664e-13  8.64470524e-14
  -1.65664531e+00  1.70164611e-01 -1.77371096e-01 -8.41902221e-01
  -2.49584748e-01 -6.89310747e-13  2.71174726e-02 -5.74163521e-04
   3.09960076e-03  5.99216382e-01  7.34788272e-03 -8.25332187e-02
  -8.07798196e-02  4.20275317e-13 -1.40125260e-01  2.71364812e-01]
 [ 1.69357942e-01  6.37355751e-13 -3.98771336e-13  3.33004382e-01
   6.93134955e-03  9.93712916e-01 -2.77588187e-12 -6.85624122e-13
   2.03536174e+00  2.95455444e-01  1.03329986e+00  1.93164638e+00
   3.338

In [10]:
weights, nto = tdhf_h2o.get_nto(state=3)
print(nto)

[[-5.28929909e-14 -7.24593057e-14  2.50924029e-01 -3.89524922e-01
   9.20628674e-01  5.86579593e-14 -6.31360924e-13  2.25289721e-14
  -7.10960994e-14 -1.19676511e-03 -6.94693641e-02 -2.92191893e-01
   4.96422582e-04  2.68321666e-02 -3.06170246e-04 -7.08942804e-02
  -3.68686418e-02  1.68934617e-01  3.99079291e-05 -3.15778906e-03
  -1.18967489e-02  1.03327633e-01  1.27143375e-04  1.03554363e-01]
 [-1.09239329e-13 -9.44989701e-14  5.44381898e-01  1.19503556e-01
   3.88555303e-02  1.10478807e-13 -9.50102933e-13  1.00519777e-12
  -9.79155092e-12  1.12832378e-01 -1.83755182e-01 -1.72039235e+00
   1.41955732e-02  3.18325562e-02 -8.32992250e-03 -4.62424290e-01
  -9.80320348e-02  8.63736163e-01  9.65231663e-04 -6.32618641e-03
  -3.90271790e-02  1.07734439e-01  3.56377159e-03  3.65311178e-01]
 [-1.76866115e-13 -8.70517389e-14  3.26367207e-01  1.53249783e-01
   9.80868237e-02  4.65696332e-13 -4.71954434e-12 -5.28838972e-12
  -4.54399147e-12  2.66466446e-02 -4.24010676e-01  2.98505860e+00
   1.110

## Full Configuration Interaction

2. Add random noise to the 

In [11]:
from pyscf import gto, scf, cc

for r in (1.0, 1.1, 1.2):
    mol = gto.M(atom=f"H 0 0 0; H 0 0 {r}")
    mf = scf.RHF(mol).run()
    mycc = cc.CCSD(mf).run()
    print(mycc.e_tot)

converged SCF energy = -1.06610864931794
E(CCSD) = -1.101150330244479  E_corr = -0.03504168092654206
-1.1011503302444787
converged SCF energy = -1.03653887502918
E(CCSD) = -1.079192945018469  E_corr = -0.0426540699892894
-1.0791929450184694
converged SCF energy = -1.00510670656849
E(CCSD) = -1.056740746497147  E_corr = -0.05163403992865674
-1.0567407464971468


In [12]:
from pyscf import mcscf
casci_h2o = mcscf.CASCI(rhf_h2o, 6, 8)
e_casci = casci_h2o.kernel()[0]
casscf_h2o = mcscf.CASSCF(rhf_h2o, 6, 8)
e_casscf = casscf_h2o.kernel()[0]

CASCI E = -75.9287975779019  E(CI) = -23.7672378468262  S^2 = 0.0000000
CASSCF energy = -75.9715206719023
CASCI E = -75.9715206719023  E(CI) = -24.1441027145826  S^2 = 0.0000000


In [13]:
casscf_h2o_df = mcscf.DFCASSCF(rhf_h2o, 6, 8, auxbasis='ccpvtzfit')
casscf_h2o_df.frozen = 1 # frozen core
e_casscf_df = casscf_h2o_df.kernel()[0]

CASSCF energy = -75.9693418142378
CASCI E = -75.9693418142378  E(CI) = -23.8077776119547  S^2 = 0.0000000


In [14]:
from pyscf.geomopt.geometric_solver import optimize as geometric_opt
mol_h2o_rhf_eq = geometric_opt(rhf_h2o)
from pyscf.geomopt.berny_solver import optimize as pyberny_opt
mol_h2o_casscf_eq = pyberny_opt(casscf_h2o)


casci_h2o.state_specific_(2) # state-specific opt
casci_grad = casci_h2o.nuc_grad_method().as_scanner()
mol_h2o_casci_2nd_ex = casci_grad.optimizer().kernel()
casscf_h2o.state_average_([.25] * 4) # state-averaged opt
casscf_grad = casscf_h2o.nuc_grad_method().as_scanner()
mol_h2o_sa_casscf = casscf_grad.optimizer().kernel()

geometric-optimize called with the following command line:
/home/yizhou_chen/miniconda3/envs/pyscf-env/lib/python3.12/site-packages/ipykernel_launcher.py --f=/run/user/1000/jupyter/runtime/kernel-v364d60431e6c451d15eeb14f0c0d27bb037bf3529.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   0.926711  -0.239664   0.000000    0.000000  0.000000  0.000000
   H  -0.926711  -0.239664   0.000000    0.000000  0.000000  0.000000
converged SCF energy = -75.9243880496981
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O     0.0000000000    -0.0492247630     0.0000000000
1 H     0.0389738682     0.0246123815     0.0000000000
2 H    -0.0389738682     0.0246123815    -0.0000000000
----------------------------------------------
cycle 1: E = -75.9243880497  dE = -75.9244  norm(grad) = 0.0816856


Step    0 : Gradient = 4.716e-02/4.922e-02 (rms/max) Energy = -75.9243880497
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 1.60000e-01 5.60800e-01 5.60800e-01



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.000000   0.105207   0.000000   -0.000000  0.105207  0.000000
   H   0.837916  -0.292267  -0.000000   -0.088795 -0.052604 -0.000000
   H  -0.837916  -0.292267   0.000000    0.088795 -0.052604  0.000000
converged SCF energy = -75.9484820737586
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O     0.0000000000    -0.0642723346    -0.0000000000
1 H     0.0019466101     0.0321361673     0.0000000000
2 H    -0.0019466101     0.0321361673    -0.0000000000
----------------------------------------------
cycle 2: E = -75.9484820738  dE = -0.024094  norm(grad) = 0.0787653


Step    1 : Displace = 1.039e-01/1.052e-01 (rms/max) Trust = 1.000e-01 (=) Grad = 4.548e-02/6.427e-02 (rms/max) E (change) = -75.9484820738 (-2.409e-02) Quality = 1.810
Hessian Eigenvalues: 2.85345e-02 5.00000e-02 5.00000e-02 ... 5.00000e-02 5.60800e-01 9.56585e-01


...

Step    5 : Displace = 7.080e-03/8.671e-03 (rms/max) Trust = 3.000e-01 (+) Grad = 1.700e-04/2.082e-04 (rms/max) E (change) = -75.6875020613 (-3.293e-05) Quality = 1.105
Hessian Eigenvalues: 4.99603e-02 5.00000e-02 5.00000e-02 ... 9.82995e-02 1.33107e-01 5.60800e-01



Geometry optimization cycle 7
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.000000  -0.159852  -0.000423   -0.000000 -0.000076 -0.000454
   H   1.262889  -0.159738   0.000212    0.001150  0.000038  0.000227
   H  -1.262889  -0.159738   0.000212   -0.001150  0.000038  0.000227
converged SCF energy = -75.764919706277
CASCI E = -75.6875025484650  E(CI) = -21.8438414800190  S^2 = 2.0000000
--------- CASCI_Scanner gradients for state 0 ----------
         x                y                z
0 O     0.0000000000     0.0000031242     0.0000172617
1 H    -0.0000019179    -0.0000015621    -0.0000086308
2 H     0.0000019179    -0.0000015621    -0.0000086308
----------------------------------------------
cycle 7: E = -75.6875025485  dE = -4.87118e-07  norm(grad) = 2.16551e-05


Step    6 : Displace = 9.807e-04/1.167e-03 (rms/max) Trust = 3.000e-01 (=) Grad = 1.250e-05/1.754e-05 (rms/max) E (change) = -75.6875025485 (-4.871e-07) Quality = 1.039
Hessian Eigenvalues: 4.99603e-02 5.00000e-02 5.00000e-02 ... 9.82995e-02 1.33107e-01 5.60800e-01
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    Time elapsed since start of run_optimizer: 13.092 seconds
geometric-optimize called with the following command line:
/home/yizhou_chen/miniconda3/en


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   0.926711  -0.239664   0.000000    0.000000  0.000000  0.000000
   H  -0.926711  -0.239664   0.000000    0.000000  0.000000  0.000000
converged SCF energy = -75.9243880496981
CASSCF energy = -75.7544776612113
CASCI E = -75.7544776612113  E(CI) = -23.5905835087930  S^2 = 1.0000000
CASCI state-averaged energy = -75.7544776612113
CASCI energy for each state
  State 0 weight 0.25  E = -75.9358707129232 S^2 = 0.0000000
  State 1 weight 0.25  E = -75.7083836279783 S^2 = 2.0000000
  State 2 weight 0.25  E = -75.6918807285047 S^2 = 2.0000000
  State 3 weight 0.25  E = -75.681775575439 S^2 = 0.0000000
--------------- StateAverageCASSCF_Scanner gradients ---------------
         x                y                z
0 O     0.0000390039    -0.0690025961    -0.0000000004
1 H    -0.0963239014     0.

RuntimeError: Nuclear gradients of <pyscf.mcscf.mc1step.StateAverageCASSCF_Scanner object at 0x7ba4905d6fc0> not converged